In [28]:
import csv
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt 
random.seed(7)
filename = '/Users/Hitesh Aryan Acharya/Desktop/OnlineClass/3-1/FoDS/insurance_dataset.csv'
ds = pd.read_csv(filename)

In [29]:
mean1 = int(ds['age'].mean())
mean2 = int(ds['bmi'].mean())
mean3 = int(ds['children'].mean())
mean4 = int(ds['charges'].mean())
print(mean1, mean2, mean3, mean4)
std1 = int(ds['age'].std())
std2 = int(ds['bmi'].std())
std3 = int(ds['children'].std())
std4 = int(ds['charges'].std())
print(std1, std2, std3, std4)

39 30 1 13270
14 6 1 12110


In [30]:
ds['age'] = (ds['age']-mean1)/std1
ds['bmi'] = (ds['bmi']-mean2)/std2
ds['children'] = (ds['children']-mean3)/std3
ds['charges'] = (ds['charges']-mean4)/std4
ds.head()

,age,bmi,children,charges
0,-1.428571,-0.350000,-1.0,0.298507
1,-1.500000,0.628333,0.0,-0.953299
2,-0.785714,0.500000,2.0,-0.728368
3,-0.428571,-1.215833,-1.0,0.719609
4,-0.500000,-0.186667,-1.0,-0.776478


In [45]:
#ds = ds.sample(frac = 1)
train_ds_count = int(0.7 * len(ds))
train_ds, test_ds = ds[:train_ds_count], ds[train_ds_count:]
x0 = np.ones(train_ds_count, dtype = int)
x1, x2, x3, y_train = train_ds['age'], train_ds['bmi'], train_ds['children'], train_ds['charges']
X = np.array([x0, x1, x2, x3])
y_train = y_train.reset_index(drop=True)

def normalE():
    W_normal = np.dot(np.linalg.inv(np.dot(X, X.T)), np.dot(X, y_train))
    print(W_normal)
    
iterations = 100
learn_rate = 0.001

def gradientD():
    W_GD = np.zeros(4)
    costsGD = []
    for _ in range(iterations):    
        delEGD = np.dot(np.dot(X.T, W_GD) - y_train, X.T)
        costGD = (1/(2*train_ds_count)*np.sum(np.square(np.dot(X.T, W_GD) - y_train)))
        costsGD.append(costGD)
        W_GD -= learn_rate*delEGD
    print(W_GD)
    
p = random.randrange(0, train_ds_count)
    
def stochasticGD():
    W_SGD = np.zeros(4)
    costsSGD = []
    for _ in range(iterations):
        delESGD = np.dot(np.dot(X.T[p], W_SGD) - y_train[p], X.T[p])
        costSGD = (1/(2*train_ds_count)*np.sum(np.square(np.dot(X.T[p], W_SGD) - y_train[p])))
        costsSGD.append(costSGD)
        W_SGD -= learn_rate*delESGD
    print(W_SGD)

#normalE()
#gradientD()
#stochasticGD()